<a href="https://colab.research.google.com/github/yyyhpets2022/NLP-PICO-Classification-with-BioClinicalBERT/blob/main/NLP_PICO_BioClinicalBERT_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Environment Setup 

In [184]:
pip install -U transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 26.6 MB/s 
     |████████████████████████████████| 7.6 MB 63.3 MB/s 
     |████████████████████████████████| 182 kB 62.1 MB/s 


In [192]:
pip install -U datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 21.0 MB/s 
     |████████████████████████████████| 212 kB 45.1 MB/s 
     |████████████████████████████████| 132 kB 58.9 MB/s 
     |████████████████████████████████| 127 kB 63.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [193]:
# Necessary Imports
import json
import random
import torch
import gc
import re

import pandas as pd
import numpy as np

from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
from datasets import Dataset, DatasetDict
from sklearn.metrics import f1_score
from transformers import DataCollatorWithPadding

In [159]:
# The text is clustered in the following labels
encoded_rr = {
    "M":0,
    "O":1,
    "A":2, 
    "I":3,
    "R":4, 
    "C":5,
    "P":6,
}

#Mon essaie pour le code

In [162]:
with open("/content/PICO_dev copy.txt") as f:
  paragraph = f.read()

paragraph = paragraph.split("\n\n")

paragraph_list = [x for i in paragraph for x in [i.split(":\n")]]

case_numb = []
new_paragraph_list = []

for in_paragraph in range(len(paragraph_list)):
    label_items = paragraph_list[in_paragraph][1].split('\n')
    case_numb.append(paragraph_list[in_paragraph][0].replace("###",""))
    new_paragraph_list.append(label_items)

In [ ]:
label_list = []
initial_list = []
sentence_list = []

for place_label in new_paragraph_list:
    #print(len(place_label))
    for sentence_index, sentence_value in enumerate(place_label):
        place_label[sentence_index] = re.split(r"\|", sentence_value)
        list_3_lab_sent = place_label[sentence_index]
        label_list.append(list_3_lab_sent[0])
        initial_list.append(list_3_lab_sent[1])
        sentence_list.append(list_3_lab_sent[2])

"""for place_label in new_paragraph_list:
    #print(len(place_label))
    for sentence_index, sentence_value in enumerate(place_label):
        place_label[sentence_index] = re.split(r"\|[A-Z]\|", sentence_value)
        list_4_lab_sent = place_label[sentence_index]
        #label_list.append(list_3_lab_sent[0])
        sentence_list.append(list_4_lab_sent[1])"""

In [ ]:
initial_list

In [ ]:
# importing pandas as pd 
import pandas as pd 
  
# dictionary of lists 
dict = {'label': label_list, 'initial': initial_list, 'sentence': sentence_list} 
    
Lab_Sent_df = pd.DataFrame(dict)
    
Lab_Sent_df

In [ ]:
print(Lab_Sent_df['initial'].unique())

In [164]:
for index, row in Lab_Sent_df.iterrows():
    trial_data.append({"label": encoded_rr[row[1]] , "text":row[2]})

In [ ]:
trial_data

#Train Dataset

In [172]:
with open("/content/PICO_train.txt") as f:
  paragraph_train = f.read()

paragraph_train = paragraph_train.split("\n\n")

paragraph_list_train = [x for i in paragraph_train for x in [i.split(":\n")]]

case_numb_train = []
new_paragraph_list_train = []

for in_paragraph in range(len(paragraph_list_train)):
    label_items = paragraph_list_train[in_paragraph][1].split('\n')
    case_numb_train.append(paragraph_list_train[in_paragraph][0].replace("###",""))
    new_paragraph_list_train.append(label_items)

In [173]:
label_list_train = []
initial_list_train = []
sentence_list_train = []

for place_label in new_paragraph_list_train:
    #print(len(place_label))
    for sentence_index, sentence_value in enumerate(place_label):
        place_label[sentence_index] = re.split(r"\|", sentence_value)
        list_3_lab_sent = place_label[sentence_index]
        label_list_train.append(list_3_lab_sent[0])
        initial_list_train.append(list_3_lab_sent[1]) 
        sentence_list_train.append(list_3_lab_sent[2]) 

In [174]:
# dictionary of lists 
dict = {'label': label_list_train, 'initial': initial_list_train, 'sentence': sentence_list_train} 
    
Train_Lab_Sent_df = pd.DataFrame(dict)
    
Train_Lab_Sent_df.head()

,label,initial,sentence
0,OBJECTIVES,A,( @ ) Assess whether cardiac rehabilitation ( ...
1,OBJECTIVES,A,This article reports the results of the first ...
2,DESIGN AND SETTING,M,A pragmatic pilot randomised controlled trial ...
3,DESIGN AND SETTING,M,Descriptive statistics were used to summarise ...
4,DESIGN AND SETTING,M,Interviews and focus groups were conducted and...


In [181]:
train_data = []

for index, row in Train_Lab_Sent_df.iterrows():
    train_data.append({"label": encoded_rr[row[1]] , "text":row[2]})

#Dev Dataset

In [107]:
with open("/content/PICO_dev.txt") as f:
  paragraph_dev = f.read()

paragraph_dev = paragraph_dev.split("\n\n")

paragraph_list_dev = [x for i in paragraph_dev for x in [i.split(":\n")]]

case_numb_dev = []
new_paragraph_list_dev = []

for in_paragraph in range(len(paragraph_list_dev)):
    label_items = paragraph_list_dev[in_paragraph][1].split('\n')
    case_numb_dev.append(paragraph_list_dev[in_paragraph][0].replace("###",""))
    new_paragraph_list_dev.append(label_items)

In [175]:
label_list_dev = []
initial_list_dev = []
sentence_list_dev = []

for place_label in new_paragraph_list_dev:
    #print(len(place_label))
    for sentence_index, sentence_value in enumerate(place_label):
        place_label[sentence_index] = re.split(r"\|", sentence_value)
        list_3_lab_sent = place_label[sentence_index]
        label_list_dev.append(list_3_lab_sent[0])
        initial_list_dev.append(list_3_lab_sent[1])
        sentence_list_dev.append(list_3_lab_sent[2])

In [176]:
# dictionary of lists 
dict = {'Label': label_list_dev, 'initial': initial_list_dev, 'Sentence': sentence_list_dev} 
    
Dev_Lab_Sent_df = pd.DataFrame(dict)
    
Dev_Lab_Sent_df.head()

,Label,initial,Sentence
0,OBJECTIVE,A,To compare the results of recording enamel opa...
1,DESIGN,M,Enamel opacities on the maxillary central inci...
2,DESIGN,M,On the first occasion scoring was according to...
3,DESIGN,M,An intraoral colour slide photograph was taken...
4,SUBJECTS,P,Three hundred and twenty-five @ children who w...


In [189]:
dev_data = []

for index, row in Dev_Lab_Sent_df.iterrows():
    dev_data.append({"label": encoded_rr[row[1]] , "text":row[2]})

#Loading the Bio_ClinicalBERT Model
Getting the model ready

In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModelForSequenceClassification.from_pretrained("emilyalsentzer/Bio_ClinicalBERT", num_labels=7)

#model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

In [207]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [208]:
dev_df = Dataset.from_pandas(pd.DataFrame.from_records(dev_data))
train_df = Dataset.from_pandas(pd.DataFrame.from_records(train_data))
dataset = DatasetDict({"train":train_df,"dev":dev_df})
tokenized_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/56 [00:00<?, ?ba/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


  0%|          | 0/32 [00:00<?, ?ba/s]

In [209]:
dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 55879
    })
    dev: Dataset({
        features: ['label', 'text'],
        num_rows: 31257
    })
})

In [210]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [211]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return {'f1':f1_score(y_true=labels, y_pred=predictions,average='micro')}

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    # overwrite_output_dir=True,
    learning_rate=2e-5,
    per_device_train_batch_size=40,
    per_device_eval_batch_size=40,
    num_train_epochs=2,
    weight_decay=0.01,
    save_strategy='steps',
    save_steps=2000,
    evaluation_strategy='steps',
    eval_steps=1000,
    
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['dev'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

gc.collect()
torch.cuda.empty_cache()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running training *****
  Num examples = 55879
  Num Epochs = 2
  Instantaneous batch size per device = 40
  Total train batch size (w. parallel, distributed & accumulation) = 40
  Gradient Accumulation steps = 1
  Total optimization steps = 2794
  Number of trainable parameters = 108315655
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


Step,Training Loss,Validation Loss


Step,Training Loss,Validation Loss


#Test Dataset

In [178]:
with open("/content/PICO_test.txt") as f:
  paragraph_test = f.read()

paragraph_test = paragraph_test.split("\n\n")

paragraph_list_test = [x for i in paragraph_test for x in [i.split(":\n")]]

case_numb_test = []
new_paragraph_list_test = []

for in_paragraph in range(len(paragraph_list_test)):
    label_items = paragraph_list_test[in_paragraph][1].split('\n')
    case_numb_test.append(paragraph_list_test[in_paragraph][0].replace("###",""))
    new_paragraph_list_test.append(label_items)

In [179]:
label_list_test = []
initial_list_test = []
sentence_list_test = []

for place_label in new_paragraph_list_test:
    #print(len(place_label))
    for sentence_index, sentence_value in enumerate(place_label):
        place_label[sentence_index] = re.split(r"\|", sentence_value)
        list_3_lab_sent = place_label[sentence_index]
        label_list_test.append(list_3_lab_sent[0])
        initial_list_test.append(list_3_lab_sent[1])
        sentence_list_test.append(list_3_lab_sent[2])

In [ ]:
# dictionary of lists 
dict = {'Label': label_list_test, 'initial': initial_list_test, 'Sentence': sentence_list_test} 
    
Test_Lab_Sent_df = pd.DataFrame(dict)
    
Test_Lab_Sent_df.head()

In [ ]:
test_data = []

for index, row in Test_Lab_Sent_df.iterrows():
    test_data.append({"label": encoded_rr[row[1]] , "text":row[2]})